# Accelerating Deals: Data-Driven Analysis of eBay Kleinanzeigen Used Cars

### Introduction

This project involves a comprehensive analysis of a used car dataset to uncover key insights into pricing patterns and influencing factors. The objective was to explore how various attributes, such as brand, model, mileage, and damage status, affect car prices, and to identify prevalent trends and anomalies within the dataset.

The analysis was carried out in several stages, including data cleaning and transformation, such as translating categorical values from German to English and standardizing date formats. We then conducted in-depth statistical analyses to examine the impact of different features on pricing, identified the most common brand and model combinations, and assessed how unrepaired damage influences vehicle prices.

Notable findings from the analysis include a significant price reduction for cars with unrepaired damage compared to those without, highlighting the substantial impact of vehicle condition on pricing. Additionally, the study revealed prevalent brand and model trends, offering valuable insights into the used car market and assisting stakeholders in making informed decisions.

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd

# Read the dataset into a pandas DataFrame
# The encoding 'latin1' is used to handle special characters in the dataset
autos = pd.read_csv('autos.csv', encoding='latin1')

In [2]:
# Display the columns of the DataFrame to understand the initial structure
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [3]:
# Rename the columns to have consistent formatting (snake_case) and more descriptive names
new_columns = [
    'date_crawled',          # Date when the ad was first crawled
    'name',                  # Name of the car
    'seller',                # Seller type (private or dealer)
    'offer_type',            # Type of listing
    'price',                 # Selling price of the car
    'abtest',                # A/B test inclusion status
    'vehicle_type',          # Type of vehicle
    'registration_year',     # Year of car registration
    'gearbox',               # Type of transmission
    'power_ps',              # Car's power in PS
    'model',                 # Car model name
    'odometer',              # Kilometers driven by the car
    'registration_month',    # Month of car registration
    'fuel_type',             # Type of fuel used by the car
    'brand',                 # Brand of the car
    'unrepaired_damage',     # Unrepaired damage status
    'ad_created',            # Date when the ad was created
    'num_of_pictures',       # Number of pictures in the ad
    'postal_code',           # Postal code for the car's location
    'last_seen'              # Last time the ad was seen online
]

# Assign the new column names to the DataFrame
autos.columns = new_columns

### Loading the Dataset and Column Renaming

In this project, we are analyzing a dataset of used car listings from eBay Kleinanzeigen. The initial step involves loading the dataset into a pandas DataFrame, which allows for efficient data manipulation and analysis. 

To ensure consistency and readability, we have renamed the columns of the DataFrame. The original column names had mixed casing and were not always descriptive. By converting them to snake_case and using more intuitive names, we make the dataset easier to work with for both technical and non-technical users. This renaming helps clarify the content of each column and adheres to common data science conventions, facilitating better understanding and further analysis.

In [4]:
autos.describe(include = 'all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 11:37:04,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


### Initial Observations from the Dataset Description

**Columns with Mostly One Value:**

- *seller*: The 'seller' column predominantly contains the value 'privat' with 49,999 out of 50,000 entries. This column is likely to be dropped as it does not provide significant variability or useful information for our analysis.
- *offer_type*: Similar to 'seller', the 'offer_type' column has almost all entries as 'Angebot' (49,999 out of 50,000). This lack of variability makes it a candidate for removal.
- *num_of_pictures*: The 'num_of_pictures' column has all values as 0, suggesting it does not contribute meaningful information and can be dropped.

**Columns Needing More Investigation:**

- *price*: The 'price' column has unique values and a high frequency of certain entries. We need to investigate for outliers and unusual values.
- *registration_year*: This column spans from 1000 to 9999, indicating potential incorrect or outlier data that needs to be examined and possibly cleaned.
- *power_ps*: With a wide range of values from 0 to 17,700, this column requires further investigation to identify and handle any outliers or incorrect data points.
- *odometer*: The 'odometer' column has values in text format (e.g., "150,000km"), which need to be converted to numeric format for proper analysis.

**Numeric Data Stored as Text:**

- *price*: This column might contain non-numeric values or formatting issues that need to be cleaned to ensure accurate analysis.
- *odometer*: The values in this column include text (e.g., "km") and commas, which need to be stripped to convert the data into a numeric format.

By addressing these points, we can clean the dataset and prepare it for a more accurate and meaningful analysis.

In [5]:
# Remove currency symbols and commas from the 'price' column and convert it to integer
autos['price'] = autos['price'].str.replace('$', '')  # Remove dollar sign
autos['price'] = autos['price'].str.replace(',', '')  # Remove commas
autos['price'] = autos['price'].astype(int)           # Convert to integer

In [6]:
# Remove 'km' and commas from the 'odometer' column and convert it to integer
autos['odometer'] = autos['odometer'].str.replace('km', '')  # Remove 'km'
autos['odometer'] = autos['odometer'].str.replace(',', '')   # Remove commas
autos['odometer'] = autos['odometer'].astype(int)            # Convert to integer

In [7]:
# Rename the 'odometer' column to 'odometer_km' for consistency
autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)  # Rename column

### Data Cleaning: Price and Odometer Columns

To ensure the data is clean and ready for analysis, we performed the following transformations on the `price` and `odometer` columns:

1. **Price Column:**
   - Removed the dollar sign (`$`) and commas from the values. These characters are non-numeric and would prevent proper numerical analysis.
   - Converted the cleaned values to integers to facilitate numerical operations and comparisons.

2. **Odometer Column:**
   - Removed the 'km' text and commas from the values. Similar to the price column, these characters are non-numeric and need to be removed for accurate analysis.
   - Converted the cleaned values to integers for consistency and ease of analysis.

3. **Column Renaming:**
   - Renamed the `odometer` column to `odometer_km` to clearly indicate that the values represent kilometers. This ensures consistency and improves the readability of the dataset.

These changes were made to standardize the data, remove any extraneous characters, and ensure all values are in a numerical format suitable for analysis. This step is crucial for accurate data manipulation and analysis in subsequent steps.

In [8]:
autos['odometer_km'].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [9]:
autos['odometer_km'].value_counts()

odometer_km
150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: count, dtype: int64

In [10]:
autos['price'].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [11]:
autos['price'].value_counts()

price
0        1421
500       781
1500      734
2500      643
1000      639
         ... 
414         1
79933       1
5198        1
18890       1
16995       1
Name: count, Length: 2357, dtype: int64

In [12]:
autos = autos[autos['price'].between(1, 350000)]
autos['price'].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

In [13]:
# Display concise summary of the DataFrame, including the data types and non-null values
autos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48565 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   date_crawled        48565 non-null  object
 1   name                48565 non-null  object
 2   seller              48565 non-null  object
 3   offer_type          48565 non-null  object
 4   price               48565 non-null  int64 
 5   abtest              48565 non-null  object
 6   vehicle_type        43979 non-null  object
 7   registration_year   48565 non-null  int64 
 8   gearbox             46222 non-null  object
 9   power_ps            48565 non-null  int64 
 10  model               46107 non-null  object
 11  odometer_km         48565 non-null  int64 
 12  registration_month  48565 non-null  int64 
 13  fuel_type           44535 non-null  object
 14  brand               48565 non-null  object
 15  unrepaired_damage   39464 non-null  object
 16  ad_created          48565 n

In [14]:
# Display the first 5 rows of selected datetime columns to get an initial view of the data
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [15]:
# Extract the date part (first 10 characters) from 'date_crawled' column
date_crawled = autos['date_crawled'].str[:10]  # Keep only the date part
# Calculate the distribution of 'date_crawled' values and normalize to get proportions
date_crawled_distribution = date_crawled.value_counts(normalize=True, dropna=False)
# Sort the distribution by date for better readability
date_crawled_distribution = date_crawled_distribution.sort_index()
date_crawled_distribution

date_crawled
2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: proportion, dtype: float64

In [16]:
# Extract the date part (first 10 characters) from 'ad_created' column
ad_created = autos['ad_created'].str[:10]  # Keep only the date part
# Calculate the distribution of 'ad_created' values and normalize to get proportions
ad_created_distribution = ad_created.value_counts(normalize=True, dropna=False)
# Sort the distribution by date for better readability
ad_created_distribution = ad_created_distribution.sort_index()
ad_created_distribution

ad_created
2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038855
2016-04-04    0.036858
2016-04-05    0.011819
2016-04-06    0.003253
2016-04-07    0.001256
Name: proportion, Length: 76, dtype: float64

In [17]:
# Extract the date part (first 10 characters) from 'last_seen' column
last_seen = autos['last_seen'].str[:10]  # Keep only the date part
# Calculate the distribution of 'last_seen' values and normalize to get proportions
last_seen_distribution = last_seen.value_counts(normalize=True, dropna=False)
# Sort the distribution by date for better readability
last_seen_distribution = last_seen_distribution.sort_index()
last_seen_distribution

last_seen
2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: proportion, dtype: float64

### Analyzing Date Columns: date_crawled, ad_created, and last_seen

To understand the activity timeline of the car listings, we analyzed the `date_crawled`, `ad_created`, and `last_seen` columns. The following steps were taken:

1. **Data Overview:**
   - Used `autos.info()` to get a concise summary of the DataFrame, including data types and the number of non-null values. This helps in understanding the structure and completeness of the dataset.

2. **Initial Inspection:**
   - Displayed the first 5 rows of the `date_crawled`, `ad_created`, and `last_seen` columns to get an initial view of the datetime data. This helps in understanding the format and content of these columns.

3. **Date Extraction and Distribution:**
   - For each of the datetime columns, we extracted the date part (first 10 characters) to focus on the day-level information.
   - Calculated the distribution of these dates to understand how frequently each date appears in the dataset. The distributions were normalized to get proportions, making it easier to compare different dates.
   - Sorted the distributions by date for better readability and analysis.

By analyzing these date columns, we gain insights into when the ads were crawled, when they were created, and the last time they were seen. This helps in understanding the activity and timeline of the listings on eBay Kleinanzeigen.

In [18]:
# Analyze the 'registration_year' column to understand its distribution
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

### Observations on `registration_year`

The `registration_year` column provides information about the year in which the car was first registered. After running the `describe()` method on this column, we observe the following:

1. **Count:** There are 48,565 non-null entries in the `registration_year` column.
   
2. **Mean:** The average registration year is approximately 2004.75, suggesting that the dataset predominantly includes cars from the early 2000s.

3. **Standard Deviation:** The standard deviation is 88.64, indicating a wide range of registration years, which suggests the presence of outliers.

4. **Minimum and Maximum Values:**
   - The minimum value is 1000, which is unrealistic as cars did not exist in the year 1000. This indicates erroneous data entries.
   - The maximum value is 9999, which is also unrealistic for car registration years.

5. **Quartiles:**
   - 25th Percentile (Q1): 1999, indicating that 25% of the cars were registered before this year.
   - 50th Percentile (Median): 2004, suggesting that half of the cars were registered before this year.
   - 75th Percentile (Q3): 2008, indicating that 75% of the cars were registered before this year.

#### Conclusion

The presence of unrealistic values (1000 and 9999) suggests that the `registration_year` column contains erroneous data that need to be cleaned. Realistic car registration years typically range from the early 1900s to the current year. Thus, it will be important to filter out these outliers to ensure accurate analysis.

In [19]:
# Count the number of listings with registration years outside the 1900 - 2016 range
invalid_registration_years = autos[(autos['registration_year'] < 1900) | (autos['registration_year'] > 2016)]
invalid_registration_years_count = invalid_registration_years.shape[0]

# Display the count of invalid registration years
invalid_registration_years_count

1884

### Validating `registration_year` Data

To ensure the accuracy of our dataset, we need to validate the `registration_year` column by identifying and counting listings with unrealistic registration years. Specifically, we are looking for cars registered outside the 1900 - 2016 interval.

#### Results:

By counting the number of listings with registration years outside this range, we can assess the extent of erroneous data:

- Listings with registration years before 1900 or after 2016: 1,884

In [20]:
# Remove listings with registration years outside the 1900 - 2016 range
autos = autos[(autos['registration_year'] >= 1900) & (autos['registration_year'] <= 2016)]

# Verify the shape of the cleaned dataset
autos.shape

(46681, 20)

### Cleaning `registration_year` Data

After identifying 1,884 listings with unrealistic registration years (before 1900 or after 2016), we decided to remove these rows entirely from the dataset. This accounts for approximately 3.77% of the total entries, which is a manageable proportion to discard without significantly impacting the dataset's integrity.

#### Steps Taken:
1. Filtered the dataset to retain only the listings with registration years between 1900 and 2016.
2. Verified the shape of the cleaned dataset to ensure the rows were successfully removed.

By performing this cleaning step, we improve the overall quality and reliability of our dataset for further analysis.

In [21]:
autos['registration_year'].value_counts(normalize = True)

registration_year
2000    0.067608
2005    0.062895
1999    0.062060
2004    0.057904
2003    0.057818
          ...   
1939    0.000021
1948    0.000021
1938    0.000021
1953    0.000021
1943    0.000021
Name: proportion, Length: 78, dtype: float64

### Observations on Cleaned `registration_year` Data

After cleaning the `registration_year` column by removing rows with unrealistic registration years, we can observe the distribution of the remaining years. The `value_counts()` method with `normalize=True` gives us the proportion of listings for each registration year:

#### Key Observations:

1. **Common Registration Years:**
   - The most common registration years are from 2000 to 2005, with each year having a proportion of around 6-7%.
   - This suggests that a significant portion of the listings are relatively recent, which aligns with the average registration year we previously calculated (around 2004.75).

2. **Rare Registration Years:**
   - There are several years with very low proportions (0.000021), indicating rare occurrences. These include years like 1939, 1948, 1938, 1953, and 1943.
   - The presence of these years indicates the dataset includes some vintage cars, albeit in very small numbers.

#### Conclusion:

The cleaned `registration_year` data now shows a more realistic and meaningful distribution. Most cars in the dataset are from the late 1990s to early 2000s, with a few listings from the mid-20th century. This provides a clearer picture of the age range of the cars being listed, allowing for more accurate analysis moving forward.

In [22]:
# Get the unique values in the 'brand' column
car_brands = autos['brand'].unique()

# Get the frequency of each brand in the dataset
# This helps in understanding the distribution of car brands
autos['brand'].value_counts()

brand
volkswagen        9862
bmw               5137
opel              5022
mercedes_benz     4503
audi              4041
ford              3263
renault           2201
peugeot           1393
fiat              1197
seat               853
skoda              766
nissan             713
mazda              709
smart              661
citroen            654
toyota             593
hyundai            468
sonstige_autos     458
volvo              427
mini               409
mitsubishi         384
honda              366
kia                330
alfa_romeo         310
porsche            286
suzuki             277
chevrolet          266
chrysler           164
dacia              123
daihatsu           117
jeep               106
subaru             100
land_rover          98
saab                77
jaguar              73
daewoo              70
trabant             65
rover               62
lancia              50
lada                27
Name: count, dtype: int64

In [23]:
# Calculate the total number of entries
total_entries = autos.shape[0]

# Get the frequency of each brand
brand_counts = autos['brand'].value_counts()

# Calculate the percentage of each brand
brand_percentages = (brand_counts / total_entries) * 100

# Display the percentages to help in deciding the threshold
brand_percentages

brand
volkswagen        21.126368
bmw               11.004477
opel              10.758124
mercedes_benz      9.646323
audi               8.656627
ford               6.989996
renault            4.714980
peugeot            2.984083
fiat               2.564212
seat               1.827296
skoda              1.640925
nissan             1.527388
mazda              1.518819
smart              1.415994
citroen            1.400998
toyota             1.270324
hyundai            1.002549
sonstige_autos     0.981127
volvo              0.914719
mini               0.876159
mitsubishi         0.822604
honda              0.784045
kia                0.706926
alfa_romeo         0.664082
porsche            0.612669
suzuki             0.593389
chevrolet          0.569825
chrysler           0.351321
dacia              0.263490
daihatsu           0.250637
jeep               0.227073
subaru             0.214220
land_rover         0.209936
saab               0.164949
jaguar             0.156381
daewoo        

### Analysis of Car Brands Aggregation

Given the brand percentages, it appears that only the top six brands (Volkswagen, BMW, Opel, Mercedes-Benz, Audi, and Ford) have more than 5% of the total listings. Aggregating by brands with more than 5% would focus the analysis on these six brands, which together make up a substantial portion of the dataset (approximately 68.18%).

On the other hand, aggregating the top 20 brands would include more variety, capturing about 90% of the dataset and providing a broader view of the used car market.

Given the goal of balancing depth and breadth in the analysis, aggregating by the top 20 brands seems to be the better approach. It includes a diverse range of brands and covers the vast majority of the data, allowing for more comprehensive insights.

In [24]:
# Select the top 20 brands
top_20_brands = brand_counts.index[:20]

# Filter the dataset to include only the top 20 brands
top_20_autos = autos[autos['brand'].isin(top_20_brands)]

# Display the shape of the filtered dataset to confirm the change
top_20_autos.shape

(43330, 20)

### Aggregating Car Brands for Analysis

To make the analysis more meaningful, we decided to aggregate by the most frequent car brands in the dataset. After considering the distribution of the brands, we explored two approaches:

1. **Brands with More Than 5% of Total Listings:** This approach focuses on brands that represent a significant portion of the dataset, which includes only six brands (Volkswagen, BMW, Opel, Mercedes-Benz, Audi, and Ford) and covers approximately 68.18% of the listings.
2. **Top 20 Brands:** This approach includes a broader range of brands, capturing about 90% of the dataset.

#### Decision:

We decided to proceed with aggregating the top 20 brands. This approach allows us to:
- Capture a diverse range of popular brands.
- Include the vast majority of the data, making the analysis both comprehensive and insightful.
- Provide a broader view of the used car market while maintaining focus on the most significant segments.

By focusing on the top 20 brands, we ensure that the analysis remains relevant and covers a wide spectrum of the market.

In [25]:
# Initialize an empty dictionary to store the average prices for each brand
top_20_brands_distribution = {}

# Loop over each brand in the top 20 brands
for brand in top_20_brands:
    # Calculate the mean price of cars for the current brand and store it in the dictionary
    # Filter the DataFrame for the current brand and compute the mean of the 'price' column
    top_20_brands_distribution[brand] = top_20_autos.loc[top_20_autos['brand'] == brand, 'price'].mean()
    
# Display the dictionary with average prices for each brand
top_20_brands_distribution

{'volkswagen': 5402.410261610221,
 'bmw': 8332.820517811953,
 'opel': 2975.2419354838707,
 'mercedes_benz': 8628.450366422385,
 'audi': 9336.687453600594,
 'ford': 3749.4695065890287,
 'renault': 2474.8646069968195,
 'peugeot': 3094.0172290021537,
 'fiat': 2813.748538011696,
 'seat': 4397.230949589683,
 'skoda': 6368.0,
 'nissan': 4743.40252454418,
 'mazda': 4112.596614950635,
 'smart': 3580.2239031770046,
 'citroen': 3779.1391437308866,
 'toyota': 5167.091062394604,
 'hyundai': 5365.254273504273,
 'sonstige_autos': 12338.550218340612,
 'volvo': 4946.501170960188,
 'mini': 10613.459657701711}

### Analysis of Average Car Prices for Top 20 Brands

In the following analysis, we computed the average car prices for the top 20 brands in our dataset. The results provide insights into the average market price for vehicles from these brands. 

Here is a summary of the average prices:

- **Volkswagen**: €5,402
- **BMW**: €8,333
- **Opel**: €2,975
- **Mercedes-Benz**: €8,628
- **Audi**: €9,337
- **Ford**: €3,749
- **Renault**: €2,475
- **Peugeot**: €3,094
- **Fiat**: €2,814
- **Seat**: €4,397
- **Skoda**: €6,368
- **Nissan**: €4,743
- **Mazda**: €4,113
- **Smart**: €3,580
- **Citroen**: €3,779
- **Toyota**: €5,167
- **Hyundai**: €5,365
- **Sonstige Autos**: €12,339
- **Volvo**: €4,947
- **Mini**: €10,613

#### Observations:

1. **Premium Brands:** Brands like Audi, Mercedes-Benz, and BMW have the highest average prices, indicating that these brands are positioned in the higher-end segment of the market. Audi and Mercedes-Benz have average prices around €9,000 or more.

2. **Value Brands:** Brands such as Opel, Renault, and Fiat have the lowest average prices, suggesting these brands are generally more affordable.

3. **Outliers:** The brand 'Sonstige Autos' (which means 'Miscellaneous Cars') shows an unusually high average price of €12,339. This could be due to a smaller sample size or inclusion of high-value niche vehicles. The 'Mini' brand also shows a high average price, reflecting its premium positioning.

4. **Consistency:** Brands like Volkswagen, Ford, and Toyota have average prices that are relatively moderate, showing a balance between affordability and premium features.

This analysis helps to understand the price range and positioning of different car brands in the market, guiding potential buyers or sellers on the expected pricing trends.

In [26]:
# Define the top 6 brands for detailed analysis
top_6_brands = ['audi', 'bmw', 'mercedes_benz', 'ford', 'opel', 'volkswagen']

# Filter the DataFrame for the top 6 brands
top_6_autos = autos[autos['brand'].isin(top_6_brands)]

# Create an empty dictionary to hold the average mileage for each brand
top_6_brands_mileage = {}

# Loop over each brand in the top 6 brands
for brand in top_6_brands:
    # Calculate the mean mileage for the current brand and store it in the dictionary
    # Filter the DataFrame for the current brand and compute the mean of the 'odometer_km' column
    top_6_brands_mileage[brand] = top_6_autos.loc[top_6_autos['brand'] == brand, 'odometer_km'].mean()

# Display the dictionary with average mileage for each brand
top_6_brands_mileage

{'audi': 129157.38678544914,
 'bmw': 132572.51313996495,
 'mercedes_benz': 130788.36331334666,
 'ford': 124266.01287159056,
 'opel': 129310.0358422939,
 'volkswagen': 128707.15879132022}

In [27]:
# Display both average prices and average mileage for the top 6 brands for comparison
top_6_brands_prices = {
    'audi': 9336.687453600594,
    'bmw': 8332.820517811953,
    'mercedes_benz': 8628.450366422385,
    'ford': 3749.4695065890287,
    'opel': 2975.2419354838707,
    'volkswagen': 5402.410261610221
}
top_6_brands_prices

{'audi': 9336.687453600594,
 'bmw': 8332.820517811953,
 'mercedes_benz': 8628.450366422385,
 'ford': 3749.4695065890287,
 'opel': 2975.2419354838707,
 'volkswagen': 5402.410261610221}

In [28]:
# Convert the top_6_brands_mileage dictionary to a Pandas Series for easier manipulation
top_6_mileage_series = pd.Series(top_6_brands_mileage)

# Convert the top_6_brands_prices dictionary to a Pandas Series for easier manipulation
top_6_prices_series = pd.Series(top_6_brands_prices)

# Create a DataFrame from the mileage Series and name the column 'mean_mileage'
top_6_brands_df = pd.DataFrame(top_6_mileage_series, columns=['mean_mileage'])

# Add a new column 'mean_price' to the DataFrame from the prices Series
top_6_brands_df['mean_price'] = top_6_prices_series

# Display the DataFrame showing both average mileage and average price for each brand
top_6_brands_df

,mean_mileage,mean_price
audi,129157.386785,9336.687454
bmw,132572.513140,8332.820518
mercedes_benz,130788.363313,8628.450366
ford,124266.012872,3749.469507
opel,129310.035842,2975.241935
volkswagen,128707.158791,5402.410262


### Comparison of Average Mileage and Prices for Top 6 Car Brands

The table below presents the average mileage and average price for the top 6 car brands: Audi, BMW, Mercedes-Benz, Ford, Opel, and Volkswagen.

| Brand         | Mean Mileage (km) | Mean Price (€) |
|---------------|--------------------|----------------|
| Audi          | 129,157            | 9,337          |
| BMW           | 132,573            | 8,333          |
| Mercedes-Benz | 130,788            | 8,628          |
| Ford          | 124,266            | 3,749          |
| Opel          | 129,310            | 2,975          |
| Volkswagen    | 128,707            | 5,402          |

#### Observations:

1. **Mileage and Price Trends**:
   - **High-End Brands**: Audi, BMW, and Mercedes-Benz have higher average prices compared to Ford, Opel, and Volkswagen. Despite their higher price tags, their average mileage is relatively high, suggesting that these premium brands are not necessarily less used.
   - **Affordable Brands**: Ford and Opel, which are priced lower, have slightly lower average mileages compared to the high-end brands. This trend indicates that these brands might be priced lower due to either lower mileage or other factors such as brand positioning.
   - **Intermediate Brand**: Volkswagen's pricing and mileage fall in between the high-end and affordable brands, reflecting its mid-range market position.

2. **Price vs. Mileage Analysis**:
   - **Correlation Insight**: There is no clear direct correlation between price and mileage in this dataset. While higher-priced brands like Audi, BMW, and Mercedes-Benz have high mileages, the same is true for some of the more affordable brands. This suggests that other factors might also influence car prices besides mileage alone.

This comparison provides valuable insights into how car prices and mileages are distributed among different brands, helping to understand the market dynamics and value propositions of each brand.

### Analysis of Aggregate Data

The aggregate data for the top 6 car brands reveals significant insights into the relationship between car mileage and pricing. Audi, BMW, and Mercedes-Benz, known for their premium status, exhibit higher average prices ranging between €8,333 and €9,337. Despite their higher cost, these brands also have relatively high average mileages, reflecting that their higher prices are not solely due to lower usage. In contrast, more affordable brands like Ford and Opel have lower average prices, from €2,975 to €3,749, and slightly lower average mileages. Volkswagen occupies a middle ground with a mean price of €5,402 and average mileage of 128,707 km. This distribution indicates that while higher prices generally associate with premium brands, the mileage does not linearly correlate with price, suggesting that factors such as brand perception, vehicle condition, and market positioning also play crucial roles in pricing. Overall, the analysis highlights that mileage alone cannot fully explain the variation in car prices, underscoring the complex interplay of multiple factors in the automotive market.

In [29]:
# Display the unique values in categorical columns to identify those using German words
print(autos['seller'].unique())
print(autos['offer_type'].unique())
print(autos['abtest'].unique())
print(autos['vehicle_type'].unique())
print(autos['gearbox'].unique())
print(autos['fuel_type'].unique())
print(autos['unrepaired_damage'].unique())

['privat' 'gewerblich']
['Angebot']
['control' 'test']
['bus' 'limousine' 'kleinwagen' 'kombi' nan 'coupe' 'suv' 'cabrio'
 'andere']
['manuell' 'automatik' nan]
['lpg' 'benzin' 'diesel' nan 'cng' 'hybrid' 'elektro' 'andere']
['nein' nan 'ja']


In [30]:
# Create dictionaries for translation mappings
seller_mapping = {
    'privat': 'private',        # Mapping 'privat' to 'private'
    'gewerblich': 'commercial'  # Mapping 'gewerblich' to 'commercial'
}

offer_type_mapping = {
    'Angebot': 'offer'  # Mapping 'Angebot' to 'offer'
    # Note: 'Gesuch' (request) is not present in the dataset
}

abtest_mapping = {
    'control': 'control',  # 'control' remains 'control'
    'test': 'test'         # 'test' remains 'test'
}

vehicle_type_mapping = {
    'kleinwagen': 'small car',        # Mapping 'kleinwagen' to 'small car'
    'limousine': 'sedan',             # Mapping 'limousine' to 'sedan'
    'kombi': 'station wagon',         # Mapping 'kombi' to 'station wagon'
    'bus': 'bus',                     # 'bus' remains 'bus'
    'cabrio': 'convertible',          # Mapping 'cabrio' to 'convertible'
    'coupe': 'coupe',                 # 'coupe' remains 'coupe'
    'suv': 'suv',                     # 'suv' remains 'suv'
    'andere': 'other'                 # Mapping 'andere' to 'other'
    # Note: nan values remain unchanged
}

gearbox_mapping = {
    'manuell': 'manual',      # Mapping 'manuell' to 'manual'
    'automatik': 'automatic'  # Mapping 'automatik' to 'automatic'
    # Note: nan values remain unchanged
}

fuel_type_mapping = {
    'benzin': 'petrol',       # Mapping 'benzin' to 'petrol'
    'diesel': 'diesel',       # 'diesel' remains 'diesel'
    'lpg': 'lpg',             # 'lpg' remains 'lpg'
    'cng': 'cng',             # 'cng' remains 'cng'
    'hybrid': 'hybrid',       # 'hybrid' remains 'hybrid'
    'elektro': 'electric',    # Mapping 'elektro' to 'electric'
    'andere': 'other'         # Mapping 'andere' to 'other'
    # Note: nan values remain unchanged
}

unrepaired_damage_mapping = {
    'ja': 'yes',  # Mapping 'ja' to 'yes'
    'nein': 'no'  # Mapping 'nein' to 'no'
    # Note: nan values remain unchanged
}

# Apply the mappings to the relevant columns
autos['seller'] = autos['seller'].map(seller_mapping)  # Translate 'seller' column
autos['offer_type'] = autos['offer_type'].map(offer_type_mapping)  # Translate 'offer_type' column
autos['abtest'] = autos['abtest'].map(abtest_mapping)  # Translate 'abtest' column
autos['vehicle_type'] = autos['vehicle_type'].map(vehicle_type_mapping)  # Translate 'vehicle_type' column
autos['gearbox'] = autos['gearbox'].map(gearbox_mapping)  # Translate 'gearbox' column
autos['fuel_type'] = autos['fuel_type'].map(fuel_type_mapping)  # Translate 'fuel_type' column
autos['unrepaired_damage'] = autos['unrepaired_damage'].map(unrepaired_damage_mapping)  # Translate 'unrepaired_damage' column

# Display the transformed dataframe to check the changes
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,private,offer,5000,control,bus,2004,manual,158,andere,150000,3,lpg,peugeot,no,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,private,offer,8500,control,sedan,1997,automatic,286,7er,150000,6,petrol,bmw,no,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,private,offer,8990,test,sedan,2009,manual,102,golf,70000,7,petrol,volkswagen,no,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,private,offer,4350,control,small car,2007,automatic,71,fortwo,70000,6,petrol,smart,no,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,private,offer,1350,test,station wagon,2003,manual,0,focus,150000,7,petrol,ford,no,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


### Explanation and Analysis of Categorical Data Translation

In this section, we identified and translated the German words in several categorical columns of the dataset into English. The translations make the data more accessible and easier to understand for English speakers. Here is a summary of the translations performed:

1. **Seller**: 
   - 'privat' was translated to 'private'.
   - 'gewerblich' was translated to 'commercial'.

2. **Offer Type**: 
   - 'Angebot' was translated to 'offer'.
   - Note: The term 'Gesuch' (request) was not present in the dataset.

3. **AB Test**: 
   - 'control' and 'test' were kept as they are since they are already in English.

4. **Vehicle Type**: 
   - 'kleinwagen' was translated to 'small car'.
   - 'limousine' was translated to 'sedan'.
   - 'kombi' was translated to 'station wagon'.
   - 'bus' remained 'bus'.
   - 'cabrio' was translated to 'convertible'.
   - 'coupe' remained 'coupe'.
   - 'suv' remained 'suv'.
   - 'andere' was translated to 'other'.

5. **Gearbox**: 
   - 'manuell' was translated to 'manual'.
   - 'automatik' was translated to 'automatic'.

6. **Fuel Type**: 
   - 'benzin' was translated to 'petrol'.
   - 'diesel' remained 'diesel'.
   - 'lpg' remained 'lpg'.
   - 'cng' remained 'cng'.
   - 'hybrid' remained 'hybrid'.
   - 'elektro' was translated to 'electric'.
   - 'andere' was translated to 'other'.

7. **Unrepaired Damage**: 
   - 'ja' was translated to 'yes'.
   - 'nein' was translated to 'no'.

By applying these translations, we improved the readability and usability of the dataset. This transformation allows both technical and non-technical users to easily interpret the data, facilitating further analysis and decision-making processes. The transformed dataset now has clear and consistent categorical data, which enhances its overall quality and accessibility.

In [31]:
# Convert date columns to datetime format
autos['date_crawled'] = pd.to_datetime(autos['date_crawled'])
autos['ad_created'] = pd.to_datetime(autos['ad_created'])
autos['last_seen'] = pd.to_datetime(autos['last_seen'])

# Convert the dates to the integer format YYYYMMDD
autos['date_crawled'] = autos['date_crawled'].dt.strftime('%Y%m%d').astype(int)
autos['ad_created'] = autos['ad_created'].dt.strftime('%Y%m%d').astype(int)
autos['last_seen'] = autos['last_seen'].dt.strftime('%Y%m%d').astype(int)

# Display the transformed dataframe to check the changes
autos[['date_crawled', 'ad_created', 'last_seen']].head()

,date_crawled,ad_created,last_seen
0,20160326,20160326,20160406
1,20160404,20160404,20160406
2,20160326,20160326,20160406
3,20160312,20160312,20160315
4,20160401,20160401,20160401


### Converting Date Columns to Uniform Numeric Format

To standardize the date information across our dataset, we transformed the date columns (`date_crawled`, `ad_created`, and `last_seen`) from their original string format (e.g., "2016-03-21") to a uniform numeric format (e.g., 20160321). This conversion allows us to easily perform numerical comparisons and analyses on these dates.

Here's an overview of the steps we took:

1. **Convert date columns to datetime format**: We used the `pd.to_datetime` function to ensure that the date columns are correctly recognized as dates by the pandas library. This step is crucial for accurate formatting and manipulation.
   
2. **Convert the dates to integer format**: We formatted the dates as strings in the `YYYYMMDD` format using the `dt.strftime('%Y%m%d')` function. Then, we converted these formatted strings to integers using `astype(int)`. This transformation ensures that the dates are represented in a consistent numeric format across the dataset.

The resulting uniform numeric format simplifies date-related operations and enhances the overall consistency of the data, making it more manageable and analyzable for further analysis.

In [32]:
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_of_pictures,postal_code,last_seen
0,20160326,Peugeot_807_160_NAVTECH_ON_BOARD,private,offer,5000,control,bus,2004,manual,158,andere,150000,3,lpg,peugeot,no,20160326,0,79588,20160406
1,20160404,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,private,offer,8500,control,sedan,1997,automatic,286,7er,150000,6,petrol,bmw,no,20160404,0,71034,20160406
2,20160326,Volkswagen_Golf_1.6_United,private,offer,8990,test,sedan,2009,manual,102,golf,70000,7,petrol,volkswagen,no,20160326,0,35394,20160406
3,20160312,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,private,offer,4350,control,small car,2007,automatic,71,fortwo,70000,6,petrol,smart,no,20160312,0,33729,20160315
4,20160401,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,private,offer,1350,test,station wagon,2003,manual,0,focus,150000,7,petrol,ford,no,20160401,0,39218,20160401


In [33]:
# Split the 'name' column into 'brand' and 'model' columns
autos[['brand', 'model']] = autos['name'].str.split('_', n=1, expand=True)

# Group by 'brand' and 'model' and count the occurrences
brand_model_counts = autos.groupby(['brand', 'model']).size().reset_index(name='count')

# Sort by the count in descending order to find the most common combinations
most_common_brand_model = brand_model_counts.sort_values(by='count', ascending=False).head(10)

# Display the top 10 most common brand/model combinations
print(most_common_brand_model)

            brand          model  count
31535  Volkswagen       Golf_1.4     75
4081          BMW           316i     75
9801         Ford         Fiesta     74
33133  Volkswagen           Polo     72
4321          BMW           318i     72
4685          BMW           320i     71
19953        Opel          Corsa     68
24660     Renault         Twingo     66
31508  Volkswagen           Golf     57
20018        Opel  Corsa_1.2_16V     56


### Finding the Most Common Brand/Model Combinations

To understand the most prevalent brand and model combinations in our dataset, we performed the following steps:

1. **Splitting the `name` Column**:
   The `name` column, which contains both brand and model information, was split into two separate columns: `brand` and `model`. This was achieved using the `str.split` method with an underscore (`_`) as the delimiter.

2. **Grouping by Brand and Model**:
   We then grouped the dataset by the new `brand` and `model` columns using the `groupby` method. This allowed us to count the number of occurrences for each unique brand/model combination.

3. **Counting and Sorting**:
   After grouping, we counted the occurrences of each combination with the `size` method and reset the index to convert the result back to a DataFrame. We sorted this DataFrame by the count in descending order to identify the most common combinations.

4. **Displaying the Results**:
   Finally, we displayed the top 10 most common brand/model combinations.

This approach helps us identify the most popular cars in the dataset, which can provide valuable insights for market analysis and business decisions.

In [35]:
# Calculate the average price for cars with and without unrepaired damage
average_price_with_damage = autos[autos['unrepaired_damage'] == 'yes']['price'].mean()
average_price_without_damage = autos[autos['unrepaired_damage'] == 'no']['price'].mean()

# Compute the price difference
price_difference = average_price_without_damage - average_price_with_damage

# Print the results
print(f"Average price for cars with unrepaired damage: ${average_price_with_damage:.2f}")
print(f"Average price for cars without unrepaired damage: ${average_price_without_damage:.2f}")
print(f"Price difference: ${price_difference:.2f}")

Average price for cars with unrepaired damage: $2241.15
Average price for cars without unrepaired damage: $7164.03
Price difference: $4922.89


### Analysis of Price Differences Between Cars with and Without Unrepaired Damage

In our analysis, we compared the average prices of cars based on whether they have unrepaired damage or not. The results are summarized below:

1. **Average Price for Cars with Unrepaired Damage**:
   - **$2,241.15**: This is the average price for cars listed as having unrepaired damage.

2. **Average Price for Cars without Unrepaired Damage**:
   - **$7,164.03**: This is the average price for cars listed as not having unrepaired damage.

3. **Price Difference**:
   - **$4,922.89**: This represents the difference in average price between cars with unrepaired damage and those without.

### Interpretation

The findings reveal a significant price disparity between cars with and without unrepaired damage. On average, cars with unrepaired damage are priced approximately **$4,922.89** lower than their non-damaged counterparts. This substantial difference highlights the impact of unrepaired damage on the market value of vehicles.

### Conclusion

This analysis aimed to uncover the factors driving used car prices and identify key trends within the dataset. By translating categorical values, standardizing date formats, and conducting detailed statistical analysis, we examined the influence of attributes such as brand, model, mileage, and damage status on vehicle pricing.

Our findings highlight a significant price drop for cars with unrepaired damage, averaging $4922.89 less than their non-damaged counterparts. The analysis also identified prevalent brand and model combinations, providing insights into market trends and aiding stakeholders in making more informed decisions.